In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import logging

import numpy as np

import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

from compute import *
from defPlot import *

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, silence_logs='error', protocol = 'tcp://',
                      host = '137.129.155.67')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:8787/status,
Dashboard: http://137.129.155.67:8787/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://137.129.155.67:46789,Workers: 1
Dashboard: http://137.129.155.67:8787/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://137.129.155.67:36951,Total threads: 4
Dashboard: http://137.129.155.67:37705/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
indir_data_RAW = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_ANOMALY/OLR/'
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/OLR/'
indir_data_VAR = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/ANALYSIS/VARIANCE/ANOMALY_FILTERED/'
var_file = 'anom_OLR_brut_ERA5_3H'


In [3]:
### Open DATA
ds_RAW = xr.open_mfdataset(indir_data_RAW+'*'+var_file+'*'+str(year)+'.nc', chunks = {'time' : 1}, parallel=True)
ds_FILTER = xr.open_mfdataset(indir_data_FILTERED + '*' + str(year) + '.nc', chunks = {'time' : 1}, parallel=True)
ds_VAR = xr.open_mfdataset(indir_data_VAR + 'OLR_YEAR.nc', chunks = {'time' : 1}, parallel=True)

ds_FILTER = ds_FILTER.assign_coords(lon = (((ds_FILTER.lon + 180) % 360) - 180)).sortby('lon')
ds_VAR = ds_VAR.assign_coords(lon = (((ds_VAR.lon + 180) % 360) - 180)).sortby('lon')

In [4]:
### Compute standard deviation
ds_VAR_mean = ds_VAR.mean().compute()
ds_std_mean = ds_VAR_mean**(0.5)
ds_std = ds_VAR**(0.5)

# Compute 2D
On calcule le nombre de jours qui se situe au dessus de l'écart-type, de 2 écart-type, et de 3 ecart-type.
On doit séparer le nombre de jour dans le négatif et dans le positif, le négatif étant un indice de convection. 

In [ ]:
wave = 'OLR_Kelvin'
coeff = 1
_da_nb_jour, _da_nb_jour_N = computeDayBelowAbove_netCDF(ds_FILTER,ds_std,coeff, wave)